<a href="https://colab.research.google.com/github/ccsed/ProgettoLabAI/blob/main/Progetto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collego colab a google drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

def conta_file(directory):
    try:

        elementi = os.listdir(directory)

        file = [f for f in elementi if os.path.isfile(os.path.join(directory, f))]

        numero_file = len(file)
        return numero_file
    except Exception as e:
        print(f"Errore: {e}")
        return 0

directory_path = '/content/Labels'
print(f"Numero di file nella directory: {conta_file(directory_path)}")


Numero di file nella directory: 3401


Importo le librerie necessarie

In [3]:
!pip install rasterio

In [4]:
import os
import numpy as np
import rasterio
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
import albumentations as A
from PIL import Image
import matplotlib.pyplot as plt
import random
import skimage.io as io
from rasterio.plot import show
import torch.nn as nn
import torchvision.transforms.functional as TF
from albumentations.pytorch import ToTensorV2
import torch.optim as optim
from tqdm import tqdm
import torchvision
from osgeo import gdal
import cv2
import tifffile as tiff
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import copy

Definisco alcune directory

In [5]:
drive_dir = '/content/drive/MyDrive'
image_dir = '/content/drive/MyDrive/Data/SN6_buildings_AOI_11_Rotterdam_train/train/AOI_11_Rotterdam/SAR-Intensity'
train_path = '/content/drive/MyDrive/train.txt'
val_path = '/content/drive/MyDrive/val.txt'
test_path = '/content/drive/MyDrive/test.txt'

Media e std calcolate in Funzioni.ipynb

In [6]:
# mean = [22.79312241, 24.30391245, 19.29892107, 17.69439361]
# std = [16.42519281, 17.42876255, 14.27711349, 13.24714869]
mean = [0.0, 0.0, 0.0, 0.0]
std = [1.0, 1.0, 1.0, 1.0]

Definiamo la classe SARDataset

In [7]:
class SARDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]


        with rasterio.open(image_path) as src:
            image = src.read().transpose((1, 2, 0))
            # print(image.shape)
            image = image.astype(np.float32)

        with rasterio.open(mask_path) as src:
            mask = src.read(1).astype(np.float32)
            # print(mask.shape)
            mask[mask == 255.0] = 1.0


        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

# class SARDataset(Dataset):
#     def __init__(self, image_paths, mask_paths, transform=None):
#         self.image_paths = image_paths
#         self.mask_paths = mask_paths
#         self.transform = transform

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         image_path = self.image_paths[idx]
#         mask_path = self.mask_paths[idx]


#         image = tiff.imread(image_path)

#         mask = tiff.imread(mask_path)
#         if mask.ndim == 3:
#             mask = mask[:, :, 0]
#         mask = mask.astype(np.float32)
#         mask[mask == 255.0] = 1.0


#         if image.shape[:2] != mask.shape:
#             raise ValueError(f"Le dimensioni dell'immagine {image_path} e della maschera {mask_path} non corrispondono.")

#         if self.transform is not None:
#             augmentations = self.transform(image=image, mask=mask)
#             image = augmentations["image"]
#             mask = augmentations["mask"]

#         return image, mask

Carico i percorsi delle immagini per il training, il validation e il test set

In [8]:
with open(train_path, 'r') as f:
    train_data = [line.split() for line in f.read().splitlines()]

train_image_paths = [line[0] for line in train_data]
train_label_paths = ['/' + line[1] for line in train_data]

with open(val_path, 'r') as f:
    val_data = [line.split() for line in f.read().splitlines()]

val_image_paths = [line[0] for line in val_data]
val_label_paths = ['/' + line[1] for line in val_data]

with open(test_path, 'r') as f:
    test_data = [line.split() for line in f.read().splitlines()]

test_image_paths = [line[0] for line in test_data]
test_label_paths = ['/' + line[1] for line in test_data]

Creo il modello UNET

Classe DoubleConv

In [9]:
class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(DoubleConv, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    )

  def forward(self, x):
    return self.conv(x)



Classe UNET

In [10]:
class UNET(nn.Module):
  def __init__(
      self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
  ):
    super(UNET, self).__init__()
    self.ups = nn.ModuleList()
    self.downs = nn.ModuleList()
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    for feature in features:
      self.downs.append(DoubleConv(in_channels, feature))
      in_channels = feature

    for feature in reversed(features):
      self.ups.append(
          nn.ConvTranspose2d(
              feature*2, feature, kernel_size=2, stride=2,
          )
      )
      self.ups.append(DoubleConv(feature*2, feature))

    self.bottleneck = DoubleConv(features[-1], features[-1]*2)
    self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)
  def forward(self, x):
    skip_connections = []

    for down in self.downs:
      x = down(x)
      skip_connections.append(x)
      x = self.pool(x)

    x = self.bottleneck(x)
    skip_connections = skip_connections[::-1]

    for idx in range(0, len(self.ups), 2):
      x = self.ups[idx](x)
      skip_connection = skip_connections[idx//2]

      if x.shape != skip_connection.shape:
        x = TF.resize(x, size=skip_connection.shape[2:])

      concat_skip = torch.cat((skip_connection, x), dim=1)
      x = self.ups[idx+1](concat_skip)

    return self.final_conv(x)

Test

In [11]:
def test():
  x = torch.randn((3, 1, 160, 160))
  model = UNET(in_channels=1, out_channels=1)
  preds = model(x)
  print(preds.shape)
  print(x.shape)
  assert preds.shape == x.shape

if __name__ == "__main__":
  test()

torch.Size([3, 1, 160, 160])
torch.Size([3, 1, 160, 160])


Seed

In [12]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

Funzione per il training

In [22]:
def train_fn(loader, model, optimizer, loss_fn, scaler, txtfile):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=device)
        targets = targets.float().to(device=device)

        with torch.cuda.amp.autocast():
            predictions = model(data).squeeze(1)
            loss = loss_fn(predictions, targets)


        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loop.set_postfix(loss=loss.item())
        loss1 = loss.item()

    with open(txtfile, "a") as f:
        f.write(f"Train Loss: {loss1:.4f}\n")


Utils

In [14]:
def save_checkpoint(state, filename="/content/drive/MyDrive/checkpoint.pth.tar"):
  print("=> Saving checkpoint")
  torch.save(state, filename)

def load_checkpoint(checkpoint, model):
  print("=> Loading checkpoint")
  model.load_state_dict(checkpoint["state_dict"])

Evaluation function

In [46]:
def eval_fn(loader, model, loss_fn, txtfile, device="cuda"):
  model.eval()
  num_correct = 0
  num_pixels = 0
  dice_score = 0
  precision = 0
  recall = 0
  f1 = 0
  val_loss = 0
  num_it = 0
  with torch.no_grad():
    for x, y in loader:
      num_it += 1
      print(num_it)
      x = x.to(device)
      y = y.to(device)
      out = model(x).squeeze(1)
      val_loss += loss_fn(out, y).item()
      preds = torch.sigmoid(out)
      preds = (preds > 0.5)
      num_correct += (preds == y).sum()
      num_pixels += torch.numel(preds)
      dice_score += (2 * (preds * y).sum()) / (((preds + y) - preds*y).sum() + 1e-8)
      precision += precision_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten(), zero_division=1)
      recall += recall_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten(), zero_division=1)
      f1 += f1_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten(), zero_division=1)

  val_loss /= len(loader)
  print(f"Validation Loss: {val_loss:.4f}")
  print(f"Accuracy: {num_correct/num_pixels*100:.2f}")
  print(f"Dice score: {dice_score/len(loader)}")
  print(f"Precision: {precision/len(loader)}")
  print(f"Recall: {recall/len(loader)}")
  print(f"F1 Score: {f1/len(loader)}")

  with open(txtfile, "a") as f:
        f.write(f"Validation Loss: {val_loss:.4f}\n")
        f.write(f"Accuracy: {num_correct/num_pixels*100:.2f}\n")
        f.write(f"Dice score: {dice_score/len(loader)}\n")
        f.write(f"Precision: {precision/len(loader)}\n")
        f.write(f"Recall: {recall/len(loader)}\n")
        f.write(f"F1 Score: {f1/len(loader)}\n")
        f.write(f"\n")

  model.train()

In [37]:
def save_predictions_as_imgs(loader, model, folder, device="cuda"):
    os.makedirs(folder, exist_ok=True)
    model.eval()
    it = 1
    for idx, (x, y) in enumerate(tqdm(loader, desc="Saving predictions")):
        print(it)
        it += 1
        x = x.to(device=device)
        y = y.to(device=device).unsqueeze(1)

        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()

        combined = torch.cat((y, preds), dim=2)
        torchvision.utils.save_image(combined, f"{folder}/comparison_{idx}.png")

    model.train()


Training baseline

In [47]:
set_seed(42)

transform = A.Compose([
    A.CenterCrop(height=896, width=896),
    A.RandomCrop(height=320, width=320),
    # A.HorizontalFlip(p=0.5),
    # A.VerticalFlip(p=0.5),
    # A.Rotate(limit=90, p=0.5),
    A.Normalize(mean=mean, std=std),
    ToTensorV2(),
])

train_dataset = SARDataset(train_image_paths, train_label_paths, transform=transform)
val_dataset = SARDataset(val_image_paths, val_label_paths, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)
load_model = True
lr = 1e-4
model_dir = "/content/drive/MyDrive/model1"
step_size = 9

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNET(in_channels=4, out_channels=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=step_size)
num_epochs = 10
scaler = torch.cuda.amp.GradScaler()

if load_model:
  load_checkpoint(torch.load("/content/drive/MyDrive/checkpoint.pth.tar"), model)



=> Loading checkpoint


In [48]:
for epoch in range(num_epochs):
        print(f"epoch {epoch}")
        train_fn(train_loader, model, optimizer, criterion, scaler, model_dir + "/output/output.txt")

        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

        eval_fn(val_loader, model, criterion, model_dir + "/output/output.txt", device=device)
        if (epoch + 1)%5 == 0:
            save_predictions_as_imgs(val_loader, model, model_dir + "/images/", device=device)

        scheduler.step()

epoch 0


100%|██████████| 149/149 [02:35<00:00,  1.04s/it, loss=0.0996]


=> Saving checkpoint
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
Validation Loss: 0.1755
Accuracy: 92.98
Dice score: 0.44459089636802673
Precision: 0.6729850260197906
Recall: 0.24862065028837044
F1 Score: 0.3584446208729438
epoch 1


100%|██████████| 149/149 [03:03<00:00,  1.23s/it, loss=0.13]


=> Saving checkpoint
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
Validation Loss: 0.1753
Accuracy: 92.80
Dice score: 0.6080385446548462
Precision: 0.5762552948390441
Recall: 0.3914717054682091
F1 Score: 0.4580095362473559
epoch 2


...

Saving predictions:  95%|█████████▌| 41/43 [00:27<00:01,  1.59it/s]

42


Saving predictions:  98%|█████████▊| 42/43 [00:28<00:00,  1.60it/s]

43


Saving predictions: 100%|██████████| 43/43 [00:28<00:00,  1.51it/s]
